# WEAT Tests

In [ ]:
from weat import weat
import os
import pandas as pd

In [ ]:
names=["flower-insect","music-instruments","career-family","math-arts","science-arts","intellect-appearance"]
attributes1=[["جميل","لطيف","هادئ","سرور","سعيد","مريح","رائع","سلام"],
             ["جميل","لطيف","هادئ","سرور","سعيد","مريح","رائع","سلام"],
             ["المهنة" ,"الشركة" ,"راتب" , "المكتب" , "المؤسسة" , "الإدارة" ,"العمل" , "الوظيفة"],
             ["له","إبن","صبي","الذكور","شقيق","الجد","هو"],
             ["له","إبن","صبي","الذكور","شقيق","الجد","هو"],
             ["النضوج","المهارة","الإبداع","الخبرة","الفطنة","الحدس","الحكمة","التحليل","البراعة","الذكاء","العبقرية","المنطق"]]

attributes2=[["قتل","غاضب","إساءة","حزين","مؤلم","مخيف","مزعج","موت"],
             ["قتل","غاضب","إساءة","حزين","مؤلم","مخيف","مزعج","موت"],
             ["منزل","أبوين","أطفال","عائلة","أبناء","زواج","زفاف","أقارب","بيت"],
             ["إبنة","شقيقة","نساء","أنثى","فتاة","هي","لها"],
             ["إبنة","شقيقة","نساء","أنثى","فتاة","هي","لها"],
             ["البشاعة","البدانة","النحافة","الأناقة","الوسامة","الجاذبية","الضعف","الجمال"]]

targets1=[["زهرة","وردة","ليلك","بنفسج","ياسمين","زنبق","أقحوان"],
          ["العود","طبل","قيثارة","ناي","مزمار","بيانو","الكمان","الساكسفون","تشيلو","جيتار"],
          ["محمد","أحمد","علي","يوسف","عمر","خالد","مصطفى","أيمن","ناصر","حسام","رامي","طارق"],
          [  "الطرح","إحداثيات", "إحصاء", "معادلات","رياضيات","مجموع","أعداد","حساب", "أرقام"],
          ["الطب","علوم","فيزياء","كيمياء","رياضيات", "الابحاث", "المختبرات"],
          ["محمد","أحمد","علي","يوسف","عمر","خالد","مصطفى","أيمن","ناصر","حسام","رامي","طارق"]]

targets2=[["حشرة","نحلة","فراشة","نملة","ذبابة","بعوضة","صرصور"],
          ["سيف","بندقية","مسدس","خنجر","فأس","صاروخ","قنبلة","السكين","المدفع"],
          ["فاطمة","عائشة","نور","ليلى","سارة","مريم","هدى","رنا","داليا","زينب","ياسمين","سلمى"],
          ["لوحة","سينما","مسرح","موسيقى" ,"رقص","فنون","رواية","غناء"],
          ["لوحة","سينما","مسرح","موسيقى" ,"رقص","فنون","رواية","غناء"],
          ["فاطمة","عائشة","نور","ليلى","سارة","مريم","هدى","رنا","داليا","زينب","ياسمين","سلمى"]]


In [ ]:
def compute_weat(targets1,targets2,attributes1,attributes2,names):
    results=[]
    years=[]
    tests=[]
    
    for filename in os.listdir('./models'):
        if filename.endswith('.bin'):
            filename=os.path.join("./models",filename)
            model_path = filename
            i=0
            while(i<len(names)):
                year = (filename[-8:-4])
                years.append(year)
            
                weat_instance = weat(model_path,attributes1[i], attributes2[i],targets1[i], targets2[i], year=year)
                weat_instance.load_model()
                #weat_instance.get_grammatical_direction(masculine_nouns_file,feminine_nouns_file)
                result = weat_instance.calc_single_weat()
                print(result)
                results.append(result)
                tests.append(names[i])
                i+=1
                
            
    results_df = pd.DataFrame(results)
    results_df = results_df.replace('nan', None)
    results_df = results_df.fillna('')
    results_df['test']=tests
    results_df['year']=years
    print(results_df)
    results_df.to_csv(r'/output/weat_notdisentangled.csv', mode="a", index=False)  
                
    print(f"##### File {names} SAVED #####")

compute_weat(targets1,targets2,attributes1,attributes2,names)
          

In [9]:
def compute_weat_disentangled(targets1, targets2, attributes1, attributes2, names):
    results = []
    masculine_nouns_file = r"nouns/masculine_nouns_ar"
    feminine_nouns_file = r"nouns/feminine_nouns_ar"
    years = []
    tests = []
    
    for filename in os.listdir('./models'):
        if filename.endswith('.bin'):
            model_path = os.path.join("./models", filename)
            year = filename[-8:-4]
            
            # Create a single weat instance for this model and perform grammatical gender identification once
            base_weat_instance = weat(model_path, [], [], [], [], year=year)
            base_weat_instance.get_grammatical_direction(masculine_nouns_file, feminine_nouns_file)
            
            for i in range(len(names)):
                years.append(year)
                
                # Create a new weat instance with the grammatical direction already computed
                weat_instance = weat(model_path, attributes1[i], attributes2[i], targets1[i], targets2[i], year=year)
                weat_instance.model = base_weat_instance.model
                weat_instance.word_vectors = base_weat_instance.word_vectors
                weat_instance.word_vectors_normalized = base_weat_instance.word_vectors_normalized
                weat_instance.vocabulary = base_weat_instance.vocabulary
                weat_instance.ar_grammar_gender_direction = base_weat_instance.ar_grammar_gender_direction
                
                result = weat_instance.calc_single_weat()
                print(result)
                results.append(result)
                tests.append(names[i])
    
    results_df = pd.DataFrame(results)
    results_df = results_df.replace('nan', None)
    results_df = results_df.fillna('')
    results_df['test'] = tests
    results_df['year'] = years
    print(results_df)
    results_df.to_csv(r'/output/weat_disentangled.csv.csv', mode="a", index=False)  
                
    print(f"##### File {names} SAVED #####")


compute_weat_disentangled(targets1, targets2, attributes1, attributes2, names)